Connect securely to the database

In [27]:
import os
import pickle
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from pymongo import MongoClient

In [28]:
load_dotenv("config.env")
MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
client = MongoClient("mongodb://" + MONGO_USER + ":" + MONGO_PASSWORD + "@localhost:27017/")
database = client.rais

Find all the users

In [29]:
users = database.fitbit.distinct('id')

Reading or loading the data of daily user steps

In [30]:
if not os.path.exists("data/users_steps_daily_check.pkl"):
    steps_dataframe = pd.DataFrame(columns=["date", "steps", "id"])
    for user in tqdm(users):
        user_dataframe = pd.DataFrame(list(
            database.fitbit.find(
                {"type": "steps",
                 "id": user},
                {"data.dateTime": 1, "data.value": 1, "id": 1, "_id": 0}
            )
        ))
        user_dataframe['date'] = user_dataframe['data'].apply(lambda d: d['dateTime'])
        user_dataframe['steps'] = user_dataframe['data'].apply(lambda d: d['value'])
        user_dataframe.drop(["data"], inplace=True, axis=1)

        # convert types
        user_dataframe['steps'] = pd.to_numeric(user_dataframe['steps'])
        user_dataframe["date"] = pd.to_datetime(pd.to_datetime(user_dataframe["date"], infer_datetime_format=True).dt.date, infer_datetime_format=True)

        steps_dataframe = pd.concat([steps_dataframe, user_dataframe], axis=0, ignore_index=True)

    # group by date and then take the sum
    steps_dataframe = steps_dataframe.groupby(['id', 'date']).sum()
    steps_dataframe.reset_index(drop=False, inplace=True)
    steps_dataframe.to_pickle("data/users_steps_daily_check.pkl")

infile = open('data/users_steps_daily_check.pkl','rb')
steps_daily = pickle.load(infile)
infile.close()
steps_daily

,id,date,steps
0,621e2e8e67b776a24055b564,2021-05-24,8833
1,621e2e8e67b776a24055b564,2021-05-25,9727
2,621e2e8e67b776a24055b564,2021-05-26,8253
3,621e2e8e67b776a24055b564,2021-05-27,9015
4,621e2e8e67b776a24055b564,2021-05-28,12949
...,...,...,...
4772,621e375b67b776a240290cdc,2021-07-29,17051
4773,621e375b67b776a240290cdc,2021-07-30,9289
4774,621e375b67b776a240290cdc,2021-07-31,17269
4775,621e375b67b776a240290cdc,2021-08-01,3692


Load the data from the previous pickle file

In [31]:
infile = open('data/users_steps_daily.pkl','rb')
steps_daily_old = pickle.load(infile)
infile.close()
steps_daily_old

,id,date,steps
0,621e375b67b776a240290cdc,2021-05-24,10830
1,621e375b67b776a240290cdc,2021-05-25,10120
2,621e375b67b776a240290cdc,2021-05-26,4985
3,621e375b67b776a240290cdc,2021-05-27,14361
4,621e375b67b776a240290cdc,2021-05-28,12175
...,...,...,...
66,621e375b67b776a240290cdc,2021-07-29,17051
67,621e375b67b776a240290cdc,2021-07-30,9289
68,621e375b67b776a240290cdc,2021-07-31,17269
69,621e375b67b776a240290cdc,2021-08-01,3692


In [33]:
infile = open('data/daily_fitbit_df_unprocessed.pkl','rb')
fitbit_daily = pickle.load(infile)
infile.close()
fitbit_daily

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,0.921642,1.341772,NaN,83.0,1349.0,0.0,0.0,<30,MALE,<19
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,0.947566,1.197531,NaN,56.0,1374.0,4.0,0.0,<30,MALE,<19
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,1.015038,1.670732,NaN,85.0,1350.0,0.0,0.0,<30,MALE,<19
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,1.191729,1.588235,NaN,90.0,1282.0,0.0,0.0,<30,MALE,<19
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,1.211896,1.090909,NaN,146.0,1274.0,4.0,0.0,<30,MALE,<19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7416,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
7417,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
7418,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
7419,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
